In [1]:
%run forward_dynamics_example.ipynb
%run pid_controller.ipynb

In [3]:
cl_sys = Function('cl_system', [Kp, Kd, xS0, nd, dt], [F(xS0, pd_control_F_simpli, dt)])
cl_sys_fun = Function('cl_system', [xS0], 
                      [cl_sys(np.array([90,90,90,90,90,90]),
                               np.array([80.0,80.0,80.0,80.0,80.0,80.0]), xS0, 
                               np.array([0.5, 0.3, 0.7, 0.5, 0.1, 1.0]), T/N)])
cl_sim = cl_sys_fun.mapaccum(N)

x0 = np.array([0.0, 0.0, 0.0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) #x y z w xq yq zq u v w p q r
res = cl_sim(x0)
res.T

t = np.linspace(0, T, N+1)

import matplotlib.pyplot as plt

import math
 
def euler_from_quaternion(q):
        w, x, y, z = q
        """
        Convert a quaternion into euler angles (roll, pitch, yaw)
        roll is rotation around x in radians (counterclockwise)
        pitch is rotation around y in radians (counterclockwise)
        yaw is rotation around z in radians (counterclockwise)
        """
        t0 = +2.0 * (w * x + y * z)
        t1 = +1.0 - 2.0 * (x * x + y * y)
        roll_x = math.atan2(t0, t1)
     
        t2 = +2.0 * (w * y - z * x)
        t2 = +1.0 if t2 > +1.0 else t2
        t2 = -1.0 if t2 < -1.0 else t2
        pitch_y = math.asin(t2)
     
        t3 = +2.0 * (w * z + x * y)
        t4 = +1.0 - 2.0 * (y * y + z * z)
        yaw_z = math.atan2(t3, t4)
     
        return np.array([roll_x, pitch_y, yaw_z]) # in radians

# Create a figure and plot the data
plt.figure()
plt.plot(t, np.concatenate((np.array(x0).reshape((13,1)), res.toarray()), axis=1)[0:3,:].T)
plt.legend(['surge - x[m]', 'sway - y[m]', 'heave -z[m]'])
plt.title("position in NED coordinate")
plt.xlabel('t [s]')

plt.figure()
vfunc = np.vectorize(euler_from_quaternion, signature='(n)->(k)')
rpy_array = vfunc(np.concatenate((np.array(x0).reshape((13,1)), res.toarray()), axis=1)[3:7,:].T)
plt.plot(t, rpy_array)
plt.legend(['roll', 'pitch', 'yaw'])
plt.title("orientation in NED euler coordinate")
plt.xlabel('t [s]')

plt.figure()
plt.plot(t, np.concatenate((np.array(x0).reshape((13,1)), res.toarray()), axis=1)[7::,:].T)
plt.legend(['u', 'v', 'w', 'p', 'q', 'r'])
plt.xlabel('t[s]')
plt.title("velocity in body coordinate")
plt.show()

RuntimeError: Error in Function::call for 'cl_system' [SXFunction] at .../casadi/core/function.cpp:370:
.../casadi/core/function_internal.hpp:1632: Assertion "arg.size()==n_in_" failed:
Incorrect number of inputs: Expected 5, got 4